# 1. Colab 환경설정


In [ ]:
! python --version

Python 3.9.16


In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 19.2 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 27.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp39-cp39-linux_x86_64.whl size=680545 sha256=e1f159b7782e52151e68bfd3af075144aed853f852a2042182a6fbbb74f1eab2
  Stored in directory: /root/.cache/pip/wheels/47/17/70/b257bc53879a458c4bfcc900e89271aa8b4f19366a54bd2455
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-a3hzgyb4
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-a3hzgyb4
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


# 2. 데이터셋 전처리

In [ ]:
import pandas as pd
# chatbot_data = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/GDSC_voicephishiing/conversation_data_set.xlsx')
chatbot_data = pd.read_excel('/content/drive/MyDrive/voice_phishing/conversation_data_set.xlsx')

In [ ]:


chatbot_data.sample(n=10)

,Unnamed: 0,0,1
1119,1119,제가 아들은 뭐 아들도 조금 온순한 편이죠. 성격이 활발하면서 자기 주장을 내세우면...,0
673,673,네. 유사 사칭 주의하세요. 계좌 번호 비밀 번호 물어 보니까. 예. 본인 맞으시죠...,1
293,293,다른 경우에 본인께서 뭐 수사 거부하거나 유익하시는 부분이 있으면 부모님께서는 이번...,1
1185,1185,어 다른 한국 사람들 모두가 제일 최고로 꼽는 관광 명소긴 한데 이제 제주도도 뭐 ...,0
424,424,예 수고하십니다 여기 서울중앙지검이고요. 저는 첨단 범죄 수사 1부에서 근무하고 있...,1
234,234,예 그렇게 되시면 작업이 완료되고 나면은 정상 대출 이용 이력 남기 때문에 이제 이...,1
1668,1668,무작정 다 수명이 짧은 건 아니고요. 솔직히 다른 강아지도 말티즈는 15년 이상은 ...,0
1491,1491,아르투로는 바르셀로나 구 구단에서 계속 영입 다른 선수를 영입할 때 조건으로 내놓는...,0
922,922,저는 고등학교 1학년 때 안 좋은 일이 있었습니다. 그게 뭐냐면 그 고등학교 1학년...,0
360,360,여보세요 예 어 지금 현재에서는 그렇기 때문에 이제 그 정확히 피해자다 피의자다 이...,1


In [ ]:

data_list = []
for q, label in zip(chatbot_data.iloc[:,1], chatbot_data.iloc[:,2])  :
    data = []
    data.append(q)
    data.append(str(int(label)))

    data_list.append(data)

In [ ]:
print(data_list[0])
print(data_list[400])
print(data_list[800])
print(data_list[1200])
print(data_list[1600])
print(data_list[2000])
print(data_list[-1])

['여보세요 응 이런 사람은 알고 계세요. 건물이나 돈을 범죄 사겠다는 권고하는 과정에서 대량의 신용카드를 대포통령이 발견이 됐는데 그중 문제가 되는게 거기서 지금 명의로 된 하나은행 통장과 농협 통장에 발견이 됐어요. 그 두 개의 통장 개설에 대해서 아시는 내용 있으세요 아니요? 전혀 없어요음 그 두 개의 통장을 작년 3월 3일 경기도 광명시 철산동에서 바둑 계절을 개설된 계좌인데 어 전혀 모르셨어요음네 알겠습니다. 끝없이 통화 불편하시면 말씀을 해 주세요음 제출 계좌를 불법 현장에서 알 수 있기 때문에 동결 처리와 동시에 북구로 환수 조치했고요. 금융 운동은 제주공 결과 또 계좌번호도 확실히 명의 계정이한테 저희가 확인을 했어요. 제가 어 저에게 연락드린 이유는 어? 본인이 직접 통장을 개설하셔서 일당들에게 금전적인 대가를 받고 통장을 판매하신 건 아닌지 일단 아니면은 제 개인 정보가 유출이 되면서 명의 조정 당하실 피해자 분이신지에 대해서 저는 연락을 드린 거예요. 통장을 판매는 양봉하신 적 있으세요 아니요? 통장을 판매를 양도 하시는 것도 아니라고 보이는 하셨고도 모른다고 보기는 하셨는데 그러면은 사이에 지갑이나 핸드폰이랑 개인정보가 있을 만한 물건을 분실하신 데가 있을까요 아니요 없어요음 질문을 좀 똑바로 좀 해주세요 장난치지 마시고. 장난 아닌데요. 네, 알겠습니다. 자 그러면은 아 혹시 이런 쇼핑몰 사이트를 이용하세요 11번가나 옥션 지마켓. 라면 그냥 저희가 이걸 왜 못 잡았냐면 대부분 평소들이 이런 식품으로 사이트에서 개인 정보였지 상당히 많이 됐더라고요. 안녕! 네 저 지금 이거 조사를 어 한 두 시간 정도 할 건데 여기 괜찮으세요? 자.', '1']
['서울중앙지방검찰청에서 전화드린 거예요네네 자 현재 저희 검찰에서 김용식 주범으로 인한 금융범죄 사기단의 검거한 상태예요. 자이 검거 현장에서 대량의 신용카드가 대포통장을 압수하던 과정에 연기로 돼 이제 농협 하나는 통장 두 점이 발견되어 조석제 연락드린 거고요. 자 문의 혹시이 통장 개별 거

# 3. Train data와 Test data로 데이터 나누기

In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [ ]:

print(len(dataset_train))
print(len(dataset_test))

1645
549


# 4. KoBERT 입력 데이터로 만들기

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# 5. KoBERT 학습모델 만들기

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

# 6. KoBERT 모델 학습시키기

In [ ]:
# for e in range(num_epochs):
#     train_acc = 0.0
#     test_acc = 0.0
#     model.train()
#     for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
#         optimizer.zero_grad()
#         token_ids = token_ids.long().to(device)
#         segment_ids = segment_ids.long().to(device)
#         valid_length= valid_length
#         label = label.long().to(device)
#         out = model(token_ids, valid_length, segment_ids)
#         loss = loss_fn(out, label)
#         loss.backward()
#         torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
#         optimizer.step()
#         scheduler.step()  # Update learning rate schedule
#         train_acc += calc_accuracy(out, label)
#         if batch_id % log_interval == 0:
#             print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
#     print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
#     model.eval()
#     for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
#         token_ids = token_ids.long().to(device)
#         segment_ids = segment_ids.long().to(device)
#         valid_length= valid_length
#         label = label.long().to(device)
#         out = model(token_ids, valid_length, segment_ids)
#         test_acc += calc_accuracy(out, label)
#     print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [ ]:
# torch.save(model, "/content/drive/MyDrive/voice_phishing/conversation_model.pt")

# 7. 새로운 문장 테스트

In [ ]:
import numpy as np

In [ ]:
# model_sent = torch.load("/content/drive/MyDrive/voice_phishing/sentence_model.pt")

In [ ]:
# model_conv = torch.load("/content/drive/MyDrive/Colab Notebooks/GDSC_voicephishiing/conversation_model.pt")

In [ ]:
model_conv = torch.load("/content/drive/MyDrive/KAIST/2022winter/voice_phishing/conversation_model_res.pt")

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
def softmax(a):
    print(type(a))
    a = a / 2
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a

    return y


def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model_conv.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model_conv(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            print(logits)
            if np.argmax(logits) == 0:
                test_eval.append("일반 대화")
            elif np.argmax(logits) == 1:
                test_eval.append("보이스피싱")
            result = softmax(logits)
            
    result_list = result.tolist()
    result_list.sort()
    # print(result_list)
    # print(f">> {result_list[-1]*100}% 확률로 " + test_eval[0] + "입니다.")
    if (test_eval[0] == "일반 대화"):
      print(100 - result_list[-1]*100)
    else:
      print(result_list[-1]*100)
    
    return np.argmax(logits)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
# sentence = "사실 기획 부분데 이번에 기획에 역량 있으 이분과는 약 2개월 전부터 우연치 않게 인연이 닿았는데, 우리 팀의 많은 호감을 비춰주셔서 개인적으로 밥도 먹고 커피도 마시며 근래 많은 대화를 나눴다. 우리는 고민하는 부분에 대한 조언도 얻게 되었다. 이번 금요일 저녁. 너무나도 자연스럽게 크루원으로서 제안을 하게 되었고 합류하게 되었다."
sentence = "금융감독원입니다. 대포통장관련하여 사건이 접수되었습니다. 신분증과 주민등록번호를 입력해주세요."
print(predict(sentence))

sentence = "이건 사기입니다. 신분증을 주세요. 아래 계좌로 300만원을 입금하세요."
print(predict(sentence))

[-3.169938   3.5800767]
<class 'numpy.ndarray'>
96.69142365455627
None
[-3.1584644  3.5751367]
<class 'numpy.ndarray'>
96.6650664806366
None


In [ ]:
tp = []
fp = []
tn = []
fn = []
for d in dataset_test:
  result = predict(d[0])
  if result == 1 and int(d[1]) == 1:
    tp.append(d)
  elif result == 1 and int(d[1]) == 0:
    fp.append(d)
  elif result == 0 and int(d[1]) == 0:
    tn.append(d)
  elif result == 0 and int(d[1]) == 1:
    fn.append(d)


print(len(tp), len(fp), len(tn), len(fn))

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


[-3.5219307  3.739608 ]
<class 'numpy.ndarray'>
97.41881489753723
[ 3.4687943 -3.7797048]
<class 'numpy.ndarray'>
2.597630023956299
[ 3.477869  -3.7830267]
<class 'numpy.ndarray'>
2.581995725631714
[ 3.2346334 -3.399921 ]
<class 'numpy.ndarray'>
3.498321771621704
[-3.5223417  3.7542355]
<class 'numpy.ndarray'>
97.43764996528625
[ 3.469847  -3.7770693]
<class 'numpy.ndarray'>
2.5996387004852295
[ 3.4744413 -3.7875369]
<class 'numpy.ndarray'>
2.580636739730835
[ 3.4453049 -3.7716167]
<class 'numpy.ndarray'>
2.63788104057312
[-3.5254967  3.7477226]
<class 'numpy.ndarray'>
97.43345379829407
[ 3.4482412 -3.7482743]
<class 'numpy.ndarray'>
2.66420841217041
[ 3.4760623 -3.769295 ]
<class 'numpy.ndarray'>
2.601611614227295
[-3.536623  3.728907]
<class 'numpy.ndarray'>
97.42382168769836
[ 3.4595804 -3.77691  ]
<class 'numpy.ndarray'>
2.6128649711608887
[-3.5143905  3.7489672]
<class 'numpy.ndarray'>
97.42109179496765
[-3.4956882  3.754214 ]
<class 'numpy.ndarray'>
97.40414023399353
[ 3.4847457 

In [ ]:
accuray = (len(tp)+len(tn))/(len(tp)+len(tn)+len(fp)+len(fn))
precision = len(tp)/(len(tp)+len(fp))
recall = len(tp)/(len(tp)+len(fn))

print(accuray, precision, recall)

0.9854280510018215 0.9742489270386266 0.9912663755458515


In [ ]:


#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

KeyboardInterrupt: ignored

# 8. Explainable AI

In [ ]:
from captum_tools_vocvis import *
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

In [ ]:
PAD_IND = tok.vocab.padding_token
PAD_IND = tok.convert_tokens_to_ids(PAD_IND)
token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model,model.bert.embeddings)

In [ ]:
transform = nlp.data.BERTSentenceTransform(tok, max_seq_length = 64, pad=True, pair=False)
voc_label_dict_inverse={ele:label_cols.index(ele) for ele in label_cols}
voc_label_dict={label_cols.index(ele):ele for ele in label_cols}

In [ ]:
def forward_with_sigmoid_for_bert(input,valid_length,segment_ids):
    return torch.sigmoid(model(input,valid_length,segment_ids))

In [ ]:
def forward_for_bert(input,valid_length,segment_ids):
    return torch.nn.functional.softmax(model(input,valid_length,segment_ids),dim=1)

In [ ]:
vis_data_records_ig = []

def interpret_sentence(model, sentence, min_len = 64, label = 0, n_steps=10):
    # text = [token for token in tok.sentencepiece(sentence)]
    # if len(text) < min_len:
    #     text += ['pad'] * (min_len - len(text))
    # indexed = tok.convert_tokens_to_ids(text)
    # print(text)
    
    # 토크나이징, 시퀀스 생성
    seq_tokens=transform([sentence])
    indexed=torch.tensor(seq_tokens[0]).long()#.to(device)
    valid_length=torch.tensor(seq_tokens[1]).long().unsqueeze(0)
    segment_ids=torch.tensor(seq_tokens[2]).long().unsqueeze(0).to(device)
    sentence=[token for token in tok.sentencepiece(sentence)]
    

    with torch.no_grad():
        model.zero_grad()

    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(0)
    
    seq_length = min_len

    # predict
    pred = forward_with_sigmoid_for_bert(input_indices,valid_length,segment_ids).detach().cpu().numpy().argmax().item()
    print(forward_with_sigmoid_for_bert(input_indices,valid_length,segment_ids))
    pred_ind = round(pred)
    
    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)

    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices,\
                                           n_steps=n_steps, return_convergence_delta=True,target=label,\
                                           additional_forward_args=(valid_length,segment_ids))

    print('pred: ', Label.vocab.itos[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, sentence, pred, pred_ind, label, delta, vis_data_records_ig)


In [ ]:
def add_attributions_to_visualizer(attributions, input_text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            voc_label_dict[pred_ind], #Label.vocab.itos[pred_ind],
                            voc_label_dict[label], # Label.vocab.itos[label],
                            100, # Label.vocab.itos[1],
                            attributions.sum(),       
                            input_text,
                            delta))


In [ ]:
visualize_text(vis_data_records_ig)